In [0]:
from pyspark.sql.functions import col, when, count, desc, sum, round

In [0]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "<client_id>",
"fs.azure.account.oauth2.client.secret":"<secret_key>",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tenant-id/oauth2/token"}


dbutils.fs.mount(
source = "abfss://olympic@olyharsh.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/olympic",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/olympic"

path,name,size,modificationTime
dbfs:/mnt/olympic/gold/,gold/,0,1741056559000
dbfs:/mnt/olympic/raw/,raw/,0,1740612934000
dbfs:/mnt/olympic/transformed/,transformed/,0,1740612943000


In [0]:
athletes_all = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympic/raw/Paris/athletes.csv")

In [0]:
athletes_all.printSchema()

root
 |-- code: integer (nullable = true)
 |-- current: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- name_short: string (nullable = true)
 |-- name_tv: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- function: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- nationality_long: string (nullable = true)
 |-- nationality_code: string (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- birth_place: string (nullable = true)
 |-- birth_country: string (nullable = true)
 |-- residence_place: string (nullable = true)
 |-- residence_country: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- hobbies: string (nullable = t

In [0]:
athletes_all.select("birth_country").show(truncate=False)

+-------------+
|birth_country|
+-------------+
|Armenia      |
|Armenia      |
|NULL         |
|Armenia      |
|Armenia      |
|PEREIRA      |
|Great Britain|
|TUBAKUTA     |
|Gambia       |
|Mexico       |
|Mexico       |
|Mexico       |
|France       |
|France       |
|Mexico       |
|Mexico       |
|Mexico       |
|NULL         |
|NULL         |
|NULL         |
+-------------+
only showing top 20 rows



In [0]:
athletes_origin = athletes_all.select("name","country","birth_country")

In [0]:
athletes_origin.show()

+--------------------+--------+-------------+
|                name| country|birth_country|
+--------------------+--------+-------------+
|    ALEKSANYAN Artur| Armenia|      Armenia|
|      AMOYAN Malkhas| Armenia|      Armenia|
|     GALSTYAN Slavik| Armenia|         NULL|
|   HARUTYUNYAN Arsen| Armenia|      Armenia|
|     TEVANYAN Vazgen| Armenia|      Armenia|
|       ARENAS Lorena|Colombia|      PEREIRA|
|     McKENZIE Ashley| Jamaica|Great Britain|
|BASS BITTAYE Gina...|  Gambia|     TUBAKUTA|
|     CAMARA Ebrahima|  Gambia|       Gambia|
| RUEDA SANTOS Lizeth|  Mexico|       Mexico|
|TAPIA VIDAL Rosa ...|  Mexico|       Mexico|
|   GRAJALES Crisanto|  Mexico|       Mexico|
|    MAAROUFOU Hachim| Comoros|       France|
|        SAADI Maesha| Comoros|       France|
|      DIOSDADO Nuria|  Mexico|       Mexico|
|       JIMENEZ Joana|  Mexico|       Mexico|
|     SOBRINO Jessica|  Mexico|       Mexico|
|      ALFEREZ Regina|  Mexico|         NULL|
|   ARELLANO Fernanda|  Mexico|   

In [0]:
tokyo_athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympic/raw/Tokyo/Athletes.csv")

In [0]:
paris_nocs = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olympic/raw/Paris/nocs.csv")

In [0]:
tokyo_athletes_join = tokyo_athletes.join(paris_nocs, tokyo_athletes.NOC == paris_nocs.country_long, "left")
tokyo_athletes_join.show()

+--------------------+--------------------+-------------------+----+-------------+--------------------+-------------+----+
|                Name|                 NOC|         Discipline|code|      country|        country_long|          tag|note|
+--------------------+--------------------+-------------------+----+-------------+--------------------+-------------+----+
|     AALERUD Katrine|              Norway|       Cycling Road| NOR|       Norway|              Norway|       norway|   P|
|         ABAD Nestor|               Spain|Artistic Gymnastics| ESP|        Spain|               Spain|        spain|   P|
|   ABAGNALE Giovanni|               Italy|             Rowing| ITA|        Italy|               Italy|        italy|   P|
|      ABALDE Alberto|               Spain|         Basketball| ESP|        Spain|               Spain|        spain|   P|
|       ABALDE Tamara|               Spain|         Basketball| ESP|        Spain|               Spain|        spain|   P|
|           ABAL

In [0]:
tokyo_athletes_join.where(col("code").isNull()).show()

+-------------------+--------------+-------------------+----+-------+------------+----+----+
|               Name|           NOC|         Discipline|code|country|country_long| tag|note|
+-------------------+--------------+-------------------+----+-------+------------+----+----+
|       ADAR Yasemin|        Turkey|          Wrestling|NULL|   NULL|        NULL|NULL|NULL|
|       AGRALI Yavuz|        Turkey|          Athletics|NULL|   NULL|        NULL|NULL|NULL|
|         AKGUL Taha|        Turkey|          Wrestling|NULL|   NULL|        NULL|NULL|NULL|
|         AKGUN Omer|        Turkey|           Shooting|NULL|   NULL|        NULL|NULL|NULL|
|       AKKUS Mihrac|        Turkey|               Judo|NULL|   NULL|        NULL|NULL|NULL|
|  AKOZ Simge Sebnem|        Turkey|         Volleyball|NULL|   NULL|        NULL|NULL|NULL|
|         AKTAS Ugur|        Turkey|             Karate|NULL|   NULL|        NULL|NULL|NULL|
|     ALBAYRAK Vedat|        Turkey|               Judo|NULL|   NULL| 

In [0]:
tokyo_athletes_join.where(col("code").isNull()).select("NOC").distinct().show()

+--------------+
|           NOC|
+--------------+
|        Turkey|
|    Cape Verde|
|Czech Republic|
+--------------+



In [0]:
paris_nocs.where(col("tag") == 'turkiye').show()

+----+-------+------------+-------+----+
|code|country|country_long|    tag|note|
+----+-------+------------+-------+----+
| TUR|Türkiye|     Türkiye|turkiye|   P|
+----+-------+------------+-------+----+



In [0]:
paris_nocs = paris_nocs.withColumn("country", when(col("country") == "Türkiye", "Turkey").otherwise(col("country"))) \
       .withColumn("country_long", when(col("country_long") == "Türkiye", "Turkey").otherwise(col("country_long")))\
           .withColumn("tag", when(col("tag") == "turkiye", "turkey").otherwise(col("tag")))

In [0]:
paris_nocs.where(col("tag") == 'turkey').show()

+----+-------+------------+------+----+
|code|country|country_long|   tag|note|
+----+-------+------------+------+----+
| TUR| Turkey|      Turkey|turkey|   P|
+----+-------+------------+------+----+



In [0]:
paris_nocs.where(col("country").like('%Verde')).show()

+----+----------+------------+----------+----+
|code|   country|country_long|       tag|note|
+----+----------+------------+----------+----+
| CPV|Cabo Verde|  Cabo Verde|cabo-verde|   P|
+----+----------+------------+----------+----+



In [0]:
paris_nocs = paris_nocs.withColumn("country", when(col("country") == "Cabo Verde", "Cape Verde").otherwise(col("country"))) \
       .withColumn("country_long", when(col("country_long") == "Cabo Verde", "Cape Verde").otherwise(col("country_long")))\
           .withColumn("tag", when(col("tag") == "cabo-verde", "cape-verde").otherwise(col("tag")))

In [0]:
paris_nocs.where(col('tag') == 'cape-verde').show()

+----+----------+------------+----------+----+
|code|   country|country_long|       tag|note|
+----+----------+------------+----------+----+
| CPV|Cape Verde|  Cape Verde|cape-verde|   P|
+----+----------+------------+----------+----+



In [0]:
paris_nocs.where(col("country").like('Czech%')).show()

+----+--------------+--------------+--------------+----+
|code|       country|  country_long|           tag|note|
+----+--------------+--------------+--------------+----+
| CZE|       Czechia|       Czechia|       czechia|   P|
| TCH|Czechoslovakia|Czechoslovakia|czechoslovakia|   H|
+----+--------------+--------------+--------------+----+



In [0]:
paris_nocs = paris_nocs.withColumn("country", when(col("country") == "Czechia", "Czech Republic").otherwise(col("country"))) \
       .withColumn("country_long", when(col("country_long") == "Czechia", "Czech Republic").otherwise(col("country_long")))\
           .withColumn("tag", when(col("tag") == "czechia", "czech-republic").otherwise(col("tag")))

We can see below that there is no discripency in the data now

In [0]:
tokyo_athletes_join = tokyo_athletes.join(paris_nocs, tokyo_athletes.NOC == paris_nocs.country_long, "left")
tokyo_athletes_join.where(col("code").isNull()).select("NOC").distinct().show()

+---+
|NOC|
+---+
+---+



In [0]:
tokyo_athletes_join.show(1)

+---------------+------+------------+----+-------+------------+------+----+
|           Name|   NOC|  Discipline|code|country|country_long|   tag|note|
+---------------+------+------------+----+-------+------------+------+----+
|AALERUD Katrine|Norway|Cycling Road| NOR| Norway|      Norway|norway|   P|
+---------------+------+------------+----+-------+------------+------+----+
only showing top 1 row



In [0]:
tokyo_athletes = tokyo_athletes_join.select("Name","code","Discipline").withColumnRenamed("Name","name").withColumnRenamed("code","country_code").withColumnRenamed("Discipline","discipline")

In [0]:
tokyo_athletes.show()

+--------------------+------------+-------------------+
|                name|country_code|         discipline|
+--------------------+------------+-------------------+
|     AALERUD Katrine|         NOR|       Cycling Road|
|         ABAD Nestor|         ESP|Artistic Gymnastics|
|   ABAGNALE Giovanni|         ITA|             Rowing|
|      ABALDE Alberto|         ESP|         Basketball|
|       ABALDE Tamara|         ESP|         Basketball|
|           ABALO Luc|         FRA|           Handball|
|        ABAROA Cesar|         CHI|             Rowing|
|       ABASS Abobakr|         SUD|           Swimming|
|    ABBASALI Hamideh|         IRI|             Karate|
|       ABBASOV Islam|         AZE|          Wrestling|
|        ABBINGH Lois|         NED|           Handball|
|         ABBOT Emily|         AUS|Rhythmic Gymnastics|
|       ABBOTT Monica|         USA|  Baseball/Softball|
|ABDALLA Abubaker ...|         QAT|          Athletics|
|      ABDALLA Maryam|         EGY|  Artistic Sw

In [0]:
tokyo_athletes.coalesce(1).write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/transformed/tokyo/athletes")

In [0]:
read_ath = spark.read.format("parquet").option("header","true").option("inferSchema","true").load("/mnt/olympic/transformed/paris/athlets_all")

In [0]:
read_ath.show(1,truncate=False)

+-------+-------+----------------+------------+----------------+------+--------+------------+-------+------------+-----------+----------------+----------------+------+------+-------------+------------------------------+----------+-----------+-------------+---------------+-----------------+----------+-----------------------------+----------+----------------------------------------------------+-------------------------+--------------------------+-------------------------------+---------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------+------------------------------------------------------+------------------+------+------------+
|code   |current|name            |name_short  |name_tv         |gender|function|country_code|country|country_long|nati

In [0]:
raw_ath = spark.read.csv("/mnt/olympic/raw/Paris/athletes.csv", header=True, inferSchema=True)

In [0]:
raw_ath.where(col("birth_country").isNull()).select("name","birth_country").show(30)

+--------------------+-------------+
|                name|birth_country|
+--------------------+-------------+
|     GALSTYAN Slavik|         NULL|
|      ALFEREZ Regina|         NULL|
|   ARELLANO Fernanda|         NULL|
|      TOSCANO Pamela|         NULL|
|   GONZALEZ Itzamary|         NULL|
|   RODRIGUEZ Samanta|         NULL|
|      INZUNZA Glenda|         NULL|
|RUBAIAWI Ali Amma...|         NULL|
|      HASSAN Ibrahim|         NULL|
| HASSAN NOUR Samiyah|         NULL|
|HOUSSEIN BARKAT H...|         NULL|
|       TEJEDA Adirem|         NULL|
|     AL SAYEGH Safia|         NULL|
|         RUIZ Angela|         NULL|
|RENTERIA RENTERIA...|         NULL|
|       ABDOLI Samyar|         NULL|
|AL MARZOOQI Omar ...|         NULL|
|PEREIRA Veronica ...|         NULL|
|          MPALI Adam|         NULL|
|  AMISON Naima-Zahra|         NULL|
|SANDOVAL AYALA Ma...|         NULL|
|        ROJAS Carlos|         NULL|
| MARTINEZ WING Bruno|         NULL|
|       FALLAH Mobina|         NULL|
|

Below we can see that the code is not correctly separating the events.

In [0]:
read_ath.where(col("code") == 1532951).show()

+-------+-------+-------------+----------+-------------+------+--------+------------+--------+------------+-----------+----------------+----------------+------+------+-------------+--------------------+--------------------+-----------+-------------+---------------+-----------------+---------+-------+----------+---------+--------------------+--------------------+-------+--------------------+----+--------------------+----------+------------------+------+--------------------+
|   code|current|         name|name_short|      name_tv|gender|function|country_code| country|country_long|nationality|nationality_long|nationality_code|height|weight|  disciplines|              events|          birth_date|birth_place|birth_country|residence_place|residence_country| nickname|hobbies|occupation|education|              family|                lang|  coach|              reason|hero|           influence|philosophy|sporting_relatives|ritual|        other_sports|
+-------+-------+-------------+----------+--

Below we can see that the birthplace is correct

In [0]:
df = spark.read.csv("/mnt/olympic/raw/Paris/athletes.csv", header=True, inferSchema=True, multiLine=True, quote='"', escape='"')
df.where(col("code") == 1532951).select("name","code","birth_place").show()


+-------------+-------+-----------+
|         name|   code|birth_place|
+-------------+-------+-----------+
|ARENAS Lorena|1532951|    PEREIRA|
+-------------+-------+-----------+



In [0]:
df.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/transformed/paris/athlets_all")

In [0]:
athlete_location = df.select("code","name","country","nationality","birth_country","residence_country")

In [0]:
athlete_location.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/transformed/paris/athlete_location")

In [0]:
athlete_location.printSchema()

root
 |-- code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- birth_country: string (nullable = true)
 |-- residence_country: string (nullable = true)



Count of Athletes per Country

In [0]:
df_gold = athlete_location.groupBy("country").agg(count("code").alias("num_athletes")).orderBy(desc("num_athletes"))

In [0]:
df_gold.show()

+-------------+------------+
|      country|num_athletes|
+-------------+------------+
|United States|         619|
|       France|         601|
|    Australia|         475|
|      Germany|         457|
|        Japan|         431|
|        Spain|         401|
|        China|         398|
|        Italy|         397|
|Great Britain|         343|
|       Canada|         332|
|  Netherlands|         290|
|       Brazil|         290|
|       Poland|         226|
|  New Zealand|         208|
|      Hungary|         177|
|      Belgium|         177|
|        Egypt|         157|
|        Korea|         147|
|    Argentina|         143|
|      Ireland|         143|
+-------------+------------+
only showing top 20 rows



Count of Athletes per Birth country

In [0]:
df_gold = athlete_location.groupBy("birth_country").agg(count("code").alias("num_athletes")).orderBy(desc("num_athletes"))
df_gold.show()
df_gold.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/paris/athletes_per_birth_country")

+-------------+------------+
|birth_country|num_athletes|
+-------------+------------+
|         NULL|        1638|
|United States|         705|
|       France|         552|
|      Germany|         396|
|        Japan|         393|
|    Australia|         391|
|        China|         367|
|        Italy|         353|
|        Spain|         341|
|Great Britain|         338|
|       Canada|         290|
|       Brazil|         267|
|  Netherlands|         235|
|  New Zealand|         186|
|       Poland|         164|
|      Hungary|         148|
|      Belgium|         139|
|    Argentina|         137|
|      Ukraine|         136|
| South Africa|         123|
+-------------+------------+
only showing top 20 rows



There are athletes whose birth country is different and they are playing for different country. 

In [0]:
from pyspark.sql.functions import when, sum

df_gold = athlete_location.withColumn(
    "migrated",
    when(df["birth_country"] != df["country"], 1).otherwise(0)
).groupBy("country").agg(
    sum("migrated").alias("migrated_athletes")
).withColumnRenamed("country", "now_playing_for_country").orderBy(desc("migrated_athletes"))
df_gold.show()
df_gold.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/paris/athletes_playing_for_country")


+-----------------------+-----------------+
|now_playing_for_country|migrated_athletes|
+-----------------------+-----------------+
|              Australia|               61|
|                 France|               43|
|                 Canada|               39|
|                  Italy|               37|
|                Ireland|               36|
|                    EOR|               36|
|                  Spain|               34|
|                 Serbia|               29|
|                Germany|               28|
|            Puerto Rico|               27|
|          United States|               26|
|                    AIN|               24|
|            New Zealand|               22|
|          Great Britain|               20|
|                Nigeria|               20|
|             Azerbaijan|               19|
|                 Israel|               15|
|                Hungary|               15|
|                Morocco|               15|
|            Netherlands|       

In [0]:

df_gold = athlete_location.withColumn(
    "migrated",
    when(df["birth_country"] != df["country"], 1).otherwise(0)
).groupBy("birth_country").agg(
    sum("migrated").alias("migrated_athletes")
).withColumnRenamed("birth_country", "migrated_from").orderBy(desc("migrated_athletes"))
df_gold.show()
df_gold.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/paris/athletes_migrated_from_country")

+------------------+-----------------+
|     migrated_from|migrated_athletes|
+------------------+-----------------+
|     United States|              179|
|Russian Federation|               79|
|     Great Britain|               63|
|            France|               61|
|           Germany|               33|
|           IR Iran|               20|
|           Ukraine|               20|
|              Cuba|               20|
|         Australia|               19|
|             China|               19|
|             Kenya|               17|
|          Ethiopia|               16|
|      South Africa|               16|
|             Spain|               16|
|           Belarus|               15|
|            Canada|               12|
|             Italy|               12|
|       Switzerland|               12|
|           Belgium|               11|
|       New Zealand|               11|
+------------------+-----------------+
only showing top 20 rows



In [0]:
df_migrated = athlete_location.withColumn(
    "migrated",
    when(df["birth_country"] != df["country"], 1).otherwise(0)
)

df_gold = df_migrated.groupBy("country").agg(
    count("code").alias("total_athletes"),
    sum("migrated").alias("migrated_athletes")
).withColumn("migration_rate", round((col("migrated_athletes") / col("total_athletes")) * 100,2)).orderBy(desc("migration_rate"))
df_gold.show()
df_gold.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/paris/athletes_migration_rate")


+--------------+--------------+-----------------+--------------+
|       country|total_athletes|migrated_athletes|migration_rate|
+--------------+--------------+-----------------+--------------+
|           EOR|            37|               36|          97.3|
|         Haiti|             7|                6|         85.71|
|       Bahrain|            14|               12|         85.71|
|Cayman Islands|             4|                3|          75.0|
|           AIN|            32|               24|          75.0|
|   South Sudan|            14|                9|         64.29|
|     Palestine|             8|                5|          62.5|
|   Puerto Rico|            51|               27|         52.94|
|      Honduras|             4|                2|          50.0|
|       Comoros|             4|                2|          50.0|
|       Andorra|             2|                1|          50.0|
|    Mauritania|             2|                1|          50.0|
|American Samoa|         

In [0]:
tokyo_coaches = spark.read.parquet("/mnt/olympic/transformed/tokyo/coaches")

In [0]:
tokyo_coaches.where(col("country_code").isNull()).show()

+----+------------+----------+
|name|country_code|discipline|
+----+------------+----------+
+----+------------+----------+



In [0]:
tokyo_coaches.printSchema()

root
 |-- name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline: string (nullable = true)



In [0]:
tokyo_gender = spark.read.parquet("/mnt/olympic/transformed/tokyo/entriesgender")

In [0]:
df_gold = tokyo_gender.withColumn(
    "Female_Percentage", 
    round((col("Female") / col("Total")) * 100, 2)
).withColumn(
    "Male_Percentage", 
    round((col("Male") / col("Total")) * 100, 2)
)

df_gold.show()

df_gold.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/tokyo/gender_percentage")


+--------------------+------+----+-----+-----------------+---------------+
|          discipline|female|male|total|Female_Percentage|Male_Percentage|
+--------------------+------+----+-----+-----------------+---------------+
|      3x3 Basketball|    32|  32|   64|             50.0|           50.0|
|             Archery|    64|  64|  128|             50.0|           50.0|
| Artistic Gymnastics|    98|  98|  196|             50.0|           50.0|
|   Artistic Swimming|   105|   0|  105|            100.0|            0.0|
|           Athletics|   969|1072| 2041|            47.48|          52.52|
|           Badminton|    86|  87|  173|            49.71|          50.29|
|   Baseball/Softball|    90| 144|  234|            38.46|          61.54|
|          Basketball|   144| 144|  288|             50.0|           50.0|
|    Beach Volleyball|    48|  48|   96|             50.0|           50.0|
|              Boxing|   102| 187|  289|            35.29|          64.71|
|        Canoe Slalom|   

In [0]:
paris_gender = spark.read.parquet("/mnt/olympic/transformed/paris/entriesgender")

In [0]:
df_gold = paris_gender.withColumn(
    "Female_Percentage", 
    round((col("Female") / col("Total")) * 100, 2)
).withColumn(
    "Male_Percentage", 
    round((col("Male") / col("Total")) * 100, 2)
)

df_gold.show()

df_gold.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/paris/gender_percentage")


+--------------------+------+----+-----+-----------------+---------------+
|          discipline|female|male|total|Female_Percentage|Male_Percentage|
+--------------------+------+----+-----+-----------------+---------------+
|              Tennis|    87|  89|  176|            49.43|          50.57|
|              Boxing|   124| 124|  248|             50.0|           50.0|
|   Marathon Swimming|    20|  18|   38|            52.63|          47.37|
|                Golf|    60|  60|  120|             50.0|           50.0|
|              Rowing|   244| 249|  493|            49.49|          50.51|
|                Judo|   186| 192|  378|            49.21|          50.79|
|             Sailing|   165| 165|  330|             50.0|           50.0|
|            Swimming|   386| 450|  836|            46.17|          53.83|
|Cycling BMX Frees...|    12|  12|   24|             50.0|           50.0|
|          Basketball|   143| 144|  287|            49.83|          50.17|
|            Handball|   

In [0]:
paris_athletes = spark.read.parquet("/mnt/olympic/transformed/paris/athletes")
paris_athletes.printSchema()

root
 |-- name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline: string (nullable = true)



In [0]:
tokyo_athletes.printSchema()

root
 |-- name: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- discipline: string (nullable = true)



In [0]:
df_joined = paris_athletes.alias("df1").join(tokyo_athletes.alias("df2"), (paris_athletes["name"] == tokyo_athletes["name"]) & (paris_athletes["country_code"] == tokyo_athletes["country_code"]), "inner").select("df1.name", "df1.country_code","df1.Discipline", "df2.Discipline")

df_joined.show()

df.joined.write.mode("overwrite").option("header",'true').parquet("/mnt/olympic/gold/paris/repeating_athletes")

+--------------------+------------+-------------------+--------------------+------------+-------------------+
|                name|country_code|         discipline|                name|country_code|         discipline|
+--------------------+------------+-------------------+--------------------+------------+-------------------+
|         ABAD Nestor|         ESP|Artistic Gymnastics|         ABAD Nestor|         ESP|Artistic Gymnastics|
|   ABAGNALE Giovanni|         ITA|             Rowing|   ABAGNALE Giovanni|         ITA|             Rowing|
|        ABAROA Cesar|         CHI|             Rowing|        ABAROA Cesar|         CHI|             Rowing|
|        ABBINGH Lois|         NED|           Handball|        ABBINGH Lois|         NED|           Handball|
|ABDALLA Abubaker ...|         QAT|          Athletics|ABDALLA Abubaker ...|         QAT|          Athletics|
|    ABDELRAHMAN Ihab|         EGY|          Athletics|    ABDELRAHMAN Ihab|         EGY|          Athletics|
|         

In [0]:
tokyo_teams = spark.read.parquet("/mnt/olympic/transformed/tokyo/teams")

In [0]:
tokyo_teams.printSchema()

root
 |-- name: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- noc: string (nullable = true)
 |-- event: string (nullable = true)

